# Object Extraction
Create a template SPARQL query for finding the lifted pre-conditions and effects of a user (NL) query.
   1. Extract the object from the user query (grounded version). Ex. Extract “unstack b2 b1” as the object.
   2. Get the pre-conditions and effects of the lifted version of the object. Ex. Finding the details of “unstack”.

## 1. Extract Grounded Objects from User Queries 

Given a user natural language (NL) query, extract the grounded object(s) from the user query. Some examples: 

- extract “moveLeft x1 y1” from the NL query “Why is moveLeft x1 y1 not used in the plan?”.
- extract “moveLeft x1 y1” from the NL query “Why is moveLeft x1 y1 used in the plan?”.
- extract both “moveLeft x1 y1” and “moveRight x1 y1” from the NL query “Why is moveLeft x1 y1 used rather than moveRight x1 y1?”.

Possible grounded objects: moveright, moveleft, moveup, movedown, pushright, pushleft, pushup, pushdown

In [1]:
import re

In [5]:
def extract_grounded_objects(query):
    # Define a list of possible grounded actions
    actions = [
        "moveright", "moveleft", "moveup", "movedown",
        "pushright", "pushleft", "pushup", "pushdown"
    ]
    
    # Build a regex pattern that matches any of the actions followed by coordinates
    # The pattern is built as follows: 
        # \b asserts a word boundary.
        # ({action_pattern}) matches any of the defined actions.
        # \s+ matches one or more whitespace characters.
        # \w+ matches one or more word characters (representing the coordinates).
    action_pattern = "|".join(actions)
    pattern = rf'\b({action_pattern})\s+\w+\s+\w+\b'
    
    # Capture the whole pattern (not just the first capturing group).
    full_matches = re.finditer(pattern, query, re.IGNORECASE)
    
    # Extract and return the complete matches
    extracted_objects = [match.group(0) for match in full_matches]
    return extracted_objects

In [6]:
# Test examples
queries = [
    "Why is moveLeft x1 y1 not used in the plan?",
    "Why is moveLeft x1 y1 used in the plan?",
    "Why is moveLeft x1 y1 used rather than moveRight x1 y1?",
]

# Extract grounded objects for each query
extracted_objects = [extract_grounded_objects(query) for query in queries]

# Print results
for i, query in enumerate(queries):
    print(f"Query: '{query}'\nExtracted: {extracted_objects[i]}\n")

Query: 'Why is moveLeft x1 y1 not used in the plan?'
Extracted: ['moveLeft x1 y1']

Query: 'Why is moveLeft x1 y1 used in the plan?'
Extracted: ['moveLeft x1 y1']

Query: 'Why is moveLeft x1 y1 used rather than moveRight x1 y1?'
Extracted: ['moveLeft x1 y1', 'moveRight x1 y1']



<br>

## 2. Get Pre-conditions and Effects of Lifted Objects

In [8]:
import rdflib
from pprint import pprint

In [9]:
file_path: str = "AI-Planning-Ontology/models/plan-ontology-rdf-instances_sokoban.owl"

g = rdflib.Graph().parse(file_path, format="xml")

In [10]:
def get_query_results(query: str):
    to_return = []
    
    for r in g.query(query):
        to_return.append(r)
    
    return to_return

In [11]:
# Get all preconditions for the specified action
action = "moveup"

actions = get_query_results(
    f"""
    PREFIX planning: <https://purl.org/ai4s/ontology/planning#>
    PREFIX rdf: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?aLabel
    WHERE {{
        planning:{action} planning:hasPrecondition ?condition .
        ?condition rdf:label ?aLabel .
    }}
    """
)

print(f"Preconditions for {action}:")
pprint(actions)

Preconditions for moveup:
[(rdflib.term.Literal('(sokoban ?sokoban)'),),
 (rdflib.term.Literal('(at ?sokoban ?x)'),),
 (rdflib.term.Literal('(below ?x ?y)'),),
 (rdflib.term.Literal('(clear ?y)'),)]


In [12]:
# Get all effects for the specified action
action = "moveup"

actions = get_query_results(
    f"""
    PREFIX planning: <https://purl.org/ai4s/ontology/planning#>
    PREFIX rdf: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?aLabel
    WHERE {{
        planning:{action} planning:hasEffect ?effect .
        ?effect rdf:label ?aLabel .
    }}
    """
)

print(f"Effects for {action}:")
pprint(actions)

Effects for moveup:
[(rdflib.term.Literal('(at ?sokoban ?y)'),),
 (rdflib.term.Literal('(clear ?x)'),),
 (rdflib.term.Literal('(not (at ?sokoban ?x))'),),
 (rdflib.term.Literal('(not (clear ?y))'),)]
